In [1]:
import random
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import dash
import plotly.graph_objects as go
from dash import html, dcc, ctx, Dash
from jupyter_dash import JupyterDash
from dash.dependencies import Output, Input
import plotly
import random
import plotly.express as px
import time
from intensity_plotter.DirectoryReplay import DirectoryReplay

In [17]:
replay = DirectoryReplay("data/source", range=(0,12))

In [18]:
generator = replay.fetchCurvesLoop(number_of_curves=10, instantly=True)

In [20]:

data_directory = "data/source"

number_of_curves = 10
drop_first = 10
drop_last = 10
cut_values_below = 0.75

## Helper functions

In [21]:
import pandas as pd
import numpy as np

def get_trace(datafile_path, time=11):
    # read data from single .dat file
    data = np.loadtxt(datafile_path)
    y_data = data[:,1][data[:,1] > cut_values_below][drop_last:-drop_first]
    x_data = data[:,0][data[:,1] > cut_values_below][drop_last:-drop_first]
    y_data[-1] = None
    x_data[-1] = None
    
    new_curve = pd.DataFrame.from_dict({
        'I': x_data,
        'q': y_data,
        't': [time]*(len(x_data))
    })
    
    return new_curve

In [23]:
# !pip install dash-core-components --upgrade
# !pip install dash_daq
# !pip install dash-latex

In [26]:
import plotly.graph_objects as go

def figure_function(data_x, 
                    data_y, 
                    data_z, 
                    scale='linear', 
                    perspective='perspective',
                    graph_style='lines'):
    data = go.Scatter3d(
    x=data_x,
    y=data_y,
    z=data_z,
    mode=graph_style,
    marker=dict(
        color=data_z,
        size=1.5,
    ),
    line=dict(
        color=data_z,
    ),
    )

    #mode = 'log' # log or linear

    layout = go.Layout(
        title='3D Scatter Plot from .dat files',
        uirevision=True,
        autosize=False,
        scene=dict(
            camera_projection_type=perspective,
            xaxis=dict(
                title='q_A^-1',
                backgroundcolor="rgb(200, 200, 230)",
                            gridcolor="white",
                            showbackground=True,
                            dtick=0.5,
                type=scale
            ),
            yaxis=dict(
                title='t',
                
                            backgroundcolor="rgb(230, 200,230)",
                            gridcolor="white",
                            showbackground=True,
                #range=[1,20],
                # rangeslider=dict(
                #     visible=True
                # )
                autorange=True,
                dtick=10,
                type=scale
                
            ),
            zaxis=dict(
                title='I',
                
                            backgroundcolor="rgb(230, 230,200)",
                            gridcolor="white",
                            showbackground=True,
                            dtick=0.5,
                            type=scale
                            
            ),
            
            aspectmode="manual", #data
            aspectratio = dict(x=2, y=5, z=1)
        ),
        #autosize=True,
        #width=800, #1200
        #height=600, #800
    )
    fig = go.Figure(data=data, layout=layout)
    fig.update_traces(connectgaps=False)

    return fig

## Main

In [27]:
import os
import dash_latex as dl

datafiles = sorted(os.listdir(data_directory))
#[k:number_of_curves+k]

X = np.random.rand(100)
Y = np.random.rand(100)
Z = np.random.rand(100)

import dash_daq as daq

# fig = go.Figure(data=[s1],layout=go.Layout(
#         xaxis=dict(range=[0, 16], autorange=False),
#         yaxis=dict(range=[0, 13], autorange=False),
#         hovermode="closest",
#         title="Start Title"
#     ))

app = Dash(__name__, external_scripts=["https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-MML-AM_CHTML" ])

app.layout = html.Div(
    [
        dcc.Graph(id='live-graph',
                  style={'height':'80vh', 'width':'100vw', },), #xaxis_title = r'$\sqrt{(n^2(t|T_))}$'
        
        dcc.Interval(
            id='graph-update',
            interval=1000,
            n_intervals=0
        ),
        # html.Button('Stop', id='stop-button', n_clicks=0),
        # html.Button('Start', id='start-button', n_clicks=0),
        daq.BooleanSwitch(
        id='update-switch',
        on=False,
        label="Pause to adjust",
        labelPosition="top"
        ),
        # html.Div(children=[html.Div([
        # html.Label(['Select Scale']), #style={'font-weight': 'bold', "text-align": "right","offset":1}
        
        # ])]),
        html.P("Scale"),
        dcc.Dropdown(
            ['log', 'linear'],
            'linear', 
            id='scale-dropdown'),
        html.P("Perspective"),
        dcc.Dropdown(
            ['perspective', 'orthographic'],
            'perspective', 
            id='perspective-dropdown'),
        html.P("Graph style"),
        dcc.Dropdown(
            ['lines', 'markers', 'lines+markers', 'text'],
            'lines', 
            id='style-dropdown'),
        html.P("Number of curves"),
        dcc.Slider(id='slider_num_curves', min=1, max=100, step=10, value=30,
                marks={x: str(x) for x in list(range(0,100,5))}),
        html.P("Interval"),
        dcc.Slider(id='slider_interval', min=100, max=2000, step=200, value=1000,
                marks={x: str(x) for x in list(range(0,2000,200))}),
    ],
        
)

@app.callback(
    Output('graph-update', "interval"),
    [Input('slider_interval', "value")]
)
def update_interval(value):
    return value    


@app.callback(Output('live-graph', 'figure'),
              [Input('graph-update', 'n_intervals')],
              #Input('start-button', 'n_clicks'),
              #Input('stop-button', 'n_clicks'))
              Input('update-switch', 'on'),
              Input('scale-dropdown', 'value'),
              Input('slider_num_curves', 'value'),
            Input('perspective-dropdown', 'value'),
            Input('style-dropdown', 'value')
            )
def update_graph_scatter(n_intervals, boolean_switch, scale, number_of_curves, perspective, style):
    if (n_intervals is None) or boolean_switch:
        return dash.no_update
    
    
    data_dataframe = pd.DataFrame(columns=["I", "q", "t"])
    data_slice = datafiles[n_intervals:number_of_curves+n_intervals]
    for i, datafile in enumerate(data_slice):
        if datafile.endswith(".dat"):
            datafile_path = os.path.join(data_directory, datafile)
            new_curve = get_trace(datafile_path, time=i)
            data_dataframe = data_dataframe.append(new_curve, ignore_index=True)
    
    X = data_dataframe['I']
    Y = data_dataframe['t']
    Z = data_dataframe['q']
    
    # scatter
    
    df = pd.DataFrame({'x': X, 'y': Y, 'z': Z})
    #fig = px.scatter_3d(df, x='x', y='y', z='z')
    # scatter = go.Scatter3d(x=df["x"], y=df["y"], z=df["z"])
    # layout=go.Layout(
    #     scene=dict(
    #         xaxis=dict(type=scale),
    #         yaxis=dict(type=scale),
    #         zaxis=dict(type=scale)
    #         ),
    #     uirevision=42) 
    # fig = go.Figure(data=scatter, layout=layout)
    fig = figure_function(X, Y, Z, scale=scale, perspective=perspective, graph_style=style)
    #fig.update(layout=go.Layout(uirevision=42))
    #print(fig.camera_eye)
    return fig

app.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8051 is in use by another program. Either identify and stop that program, or start the server with a different port.


AssertionError: 

In [6]:
!npx kill-port 8050

Process on port 8050 killed
